In [61]:
import sys
import os
# Add the root directory of the project to PYTHONPATH
sys.path.append(os.path.abspath(os.path.join('../../master-thesis-dizio-ay2324')))

In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from fairlib import DataFrame
from fairlib.preprocessing import DisparateImpactRemover
from fairlib.metrics import statistical_parity_difference, disparate_impact
from fairlib.preprocessing.lfr import LFR
random_state = 42
np.random.seed(random_state)

In [63]:
import pandas as pd

df_cleaned = pd.read_csv('cleaned_dataset.csv')

In [64]:
features = [
    # Base Attributes
    "Sex_int",
    "Protected category",
    "Overall",
    "Technical Skills",
    "Standing/Position",
    "Comunication",
    "Maturity",
    "Dynamism",
    "Mobility",
    "English",
    "Italian Residence",
    "European Residence",
    "Age Range_int",
    "number_of_searches",
    # Custom Similarity Scores
    "experience_match_score",
    "current_salary_fit_score",
    "expected_salary_fit_score",
    "study_title_score",
    "professional_similarity_score",
    "study_area_score",
    "general_similarity_score",
    "Distance Residence - Akkodis HQ",
    "Distance Residence - Assumption HQ",
]

In [65]:
protected_attributes = [
    'Sex_int', 'Protected category', 'Age Range_int',
    'Italian Residence', 'European Residence'
]

In [ ]:
# bool_cols = df_cleaned.select_dtypes(include='bool').columns
# non_bool_cols = df_cleaned.columns.difference(bool_cols)
# df_cleaned[bool_cols] = df_cleaned[bool_cols].astype(int)

# # X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=random_state)
# # scaler = StandardScaler()
# # X_train[non_bool_cols] = scaler.fit_transform(X_train[non_bool_cols])
# # X_test[non_bool_cols] = scaler.transform(X_test[non_bool_cols])

# X_train, X_test, y_train, y_test, s_train, s_test = train_test_split(
#     df_cleaned[features], df_cleaned[['Hired']], df_cleaned[protected_attributes], test_size=0.2
# )

# # Initialize the DisparateImpactRemover with desired repair level
# dir = DisparateImpactRemover(repair_level=0.8)

# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# dir.fit(X_train, s=s_train)
# # Transform both training and test data
# X_train_transformed = dir.transform(X_train, s=s_train)
# X_test_transformed = dir.transform(X_test, s=s_test)

# # Train a classifier on the transformed data
# clf = LogisticRegression()
# clf.fit(X_train_transformed, y_train)

# # Make predictions
# y_pred = clf.predict(X_test_transformed)


AttributeError: 'DisparateImpactRemover' object has no attribute 'fit'

In [66]:
def get_prepared_data(X, y, target, sensitive):
    """
    Prepare the data for training the LFR model.

    Args:
        X (DataFrame): The input features.
        y (Series): The target labels.
        target (str): The name of the target column.
        sensitive (str): The name of the sensitive attribute column.

    Returns:
        tuple: A tuple containing the prepared dataset, input features, and target labels.
    """
    X_train = X.copy()
    y_train = y.copy()
    X_train[target] = y_train
    train_dataset = DataFrame(X_train)
    train_dataset.targets = target
    train_dataset.sensitive = sensitive
    X_train.drop(columns=[target], inplace=True)
    return train_dataset, X_train, y_train

In [67]:
set(df_cleaned['Italian Residence'])

{False, True, nan}

In [68]:
df_cleaned['Italian Residence'] = df_cleaned['Italian Residence'].fillna(False)


/tmp/ipykernel_89263/3757881217.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_cleaned['Italian Residence'] = df_cleaned['Italian Residence'].fillna(False)


In [78]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

dataset = DataFrame(df_cleaned[features+['Hired']])
dataset.sensitive = "Sex_int"

bool_cols = dataset.select_dtypes(include='bool').columns
non_bool_cols = dataset[features].columns.difference(bool_cols)


dataset[bool_cols] = dataset[bool_cols].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    dataset[features], dataset[['Hired']], test_size=0.2
)
lfr = LFR(
    input_dim=len(features),  # Number of input features
    latent_dim=8,                # Dimension of fair representation
    output_dim=len(features), # Same as input for reconstruction
    alpha_z=1.0,                 # Weight for fairness loss
    alpha_x=1.0,                 # Weight for reconstruction loss
    alpha_y=1.0                  # Weight for classification loss
)
EPOCHS = 100
BATCH_SIZE = 128
TARGET = "Hired"
SENSITIVE = "Sex_int"
scaler = MinMaxScaler()
X_train[non_bool_cols] = scaler.fit_transform(X_train[non_bool_cols])
X_test[non_bool_cols] = scaler.transform(X_test[non_bool_cols])
lfr_train_dataset, X_train_copy, y_train_copy = get_prepared_data(
    X_train, y_train, TARGET, SENSITIVE
)
print(non_bool_cols)
min_max = lfr_train_dataset.agg(['min', 'max']).T
print(min_max)
lfr.fit(lfr_train_dataset, epochs=EPOCHS, batch_size=BATCH_SIZE)
# Transform data to fair representation
X_train_fair = lfr.transform(X_train)
X_test_fair = lfr.transform(X_test)

# Use transformed data with any classifier
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_train_fair, y_train)
predictions = clf.predict(X_test_fair)

Index(['Age Range_int', 'Comunication', 'Distance Residence - Akkodis HQ',
       'Distance Residence - Assumption HQ', 'Dynamism', 'English', 'Maturity',
       'Mobility', 'Overall', 'Sex_int', 'Standing/Position',
       'Technical Skills', 'current_salary_fit_score',
       'expected_salary_fit_score', 'experience_match_score',
       'general_similarity_score', 'number_of_searches',
       'professional_similarity_score', 'study_area_score',
       'study_title_score'],
      dtype='object')
                                    min  max
Sex_int                             0.0  1.0
Protected category                  0.0  1.0
Overall                             0.0  1.0
Technical Skills                    0.0  1.0
Standing/Position                   0.0  1.0
Comunication                        0.0  1.0
Maturity                            0.0  1.0
Dynamism                            0.0  1.0
Mobility                            0.0  1.0
English                             0.0  1.0
Ita

RuntimeError: all elements of input should be between 0 and 1

In [80]:
X_train.shape[1]

23

In [ ]:
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from fairlearn.metrics import (
    MetricFrame,
    demographic_parity_difference,
    equalized_odds_difference
)
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

df = df_cleaned[features+['Hired']]

protected_attributes = [
    'Sex_int', 'Protected category', 'Age Range_int',
    'Italian Residence', 'European Residence'
]

df = df.dropna(subset=protected_attributes).reset_index(drop=True)

X = df.drop(columns=['Hired'])
y = df['Hired']

bool_cols = X.select_dtypes(include='bool').columns
non_bool_cols = X.columns.difference(bool_cols)

X[bool_cols] = X[bool_cols].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=random_state)

scaler = StandardScaler()
X_train[non_bool_cols] = scaler.fit_transform(X_train[non_bool_cols])
X_test[non_bool_cols] = scaler.transform(X_test[non_bool_cols])

imputer = SimpleImputer(strategy='mean')
X_train_imp = pd.DataFrame(imputer.fit_transform(X_train), columns=X.columns)
X_test_imp = pd.DataFrame(imputer.transform(X_test), columns=X.columns)

svm_smote = SMOTE(sampling_strategy=0.75, random_state=random_state)
X_res, y_res = svm_smote.fit_resample(X_train_imp, y_train)

majority_count = (y_res == 0).sum()
minority_count = (y_res == 1).sum()

catboost_model = models['CatBoost']()
lightgbm_model = models['LightGBM']()
ensemble_model = models['Ensemble']()

catboost_model.fit(X_res, y_res)
lightgbm_model.fit(X_res, y_res)